In [ ]:
# https://docs.ultralytics.com/zh/yolov5/tutorials/train_custom_data/#21-create-datasetyaml
# https://drive.google.com/drive/folders/1svFSy2Da3cVMvekBwe13mzyx38XZ9xWo

In [9]:
import os
import xml.etree.ElementTree as ET
import shutil
from PIL import Image

classs = []
def convert_annotation(xml_file, img_width, img_height):
    try:
        tree = ET.parse(xml_file)
    except ET.ParseError as e:
        print(f"Error parsing {xml_file}: {e}")
        return []
    root = tree.getroot()
    yolo_annotations = []

    for obj in root.findall('object'):
        # 获取类别索引，减去 1 以适配 YOLOv5 的要求
        class_id = int(obj.find('name').text)
        
        # 获取目标的边界框
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # 转换为 YOLO 格式 (相对坐标)
        x_center = (xmin + xmax) / 2 / img_width
        y_center = (ymin + ymax) / 2 / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        classs.append(class_id)
        # 添加到标注列表
        yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")
    
    return yolo_annotations

def convert_dataset(image_dir, annotation_dir, output_image_dir, output_label_dir, image_set_file):
    if not os.path.exists(output_image_dir):
        os.makedirs(output_image_dir)
    if not os.path.exists(output_label_dir):
        os.makedirs(output_label_dir)
    
    with open(image_set_file, 'r') as f:
        image_names = f.read().splitlines()

    for image_name in image_names:
        img_file = image_name + '.jpg'
        img_path = os.path.join(image_dir, img_file)
        img = Image.open(img_path)
        img_width, img_height = img.size  # 获取图像宽度和高度

        # 获取 XML 文件路径
        xml_file = os.path.join(annotation_dir, image_name + '.xml')
        
        if os.path.exists(img_path) and os.path.exists(xml_file):
            # 获取目标标注
            yolo_annotations = convert_annotation(xml_file, img_width, img_height)

            # 将图像复制到输出目录
            shutil.copy(img_path, os.path.join(output_image_dir, img_file))

            # 保存为 YOLO 格式的标注文件
            txt_file = os.path.join(output_label_dir, image_name + '.txt')
            with open(txt_file, "w") as f:
                f.write("\n".join(yolo_annotations))

# 设置路径
image_dir = './dataset/Detection/VOC2007/JPEGImages'
annotation_dir = './dataset/Detection/VOC2007/Annotations'
output_image_dir_train = './IP102_YOLOv5/images/train'
output_label_dir_train = './IP102_YOLOv5/labels/train'
output_image_dir_val = './IP102_YOLOv5/images/val'
output_label_dir_val = './IP102_YOLOv5/labels/val'

# 训练集和验证集的文件路径
trainval_file = './dataset/Detection/VOC2007/ImageSets/Main/trainval.txt'
test_file = './dataset/Detection/VOC2007/ImageSets/Main/test.txt'

# 处理训练集
convert_dataset(image_dir, annotation_dir, output_image_dir_train, output_label_dir_train, trainval_file)

# 处理测试集
convert_dataset(image_dir, annotation_dir, output_image_dir_val, output_label_dir_val, test_file)

print(min(classs),max(classs))

Error parsing ./dataset/Detection/VOC2007/Annotations/IP087000986.xml: junk after document element: line 27, column 0
0 101
